In [1]:
## 设置数据库 & 定义数据模型
    
load './wtnb/wtnb.rb'
load './wtnb/wtmodel.rb'

IRuby.display IRuby.html(include_js)
IRuby.display IRuby.html(include_css)
nil

connect_db("online")
enable_cache

"<script src='https://unpkg.com/vue/dist/vue.js'></script>\n<script src='https://unpkg.com/chart.js@2.9.3/dist/Chart.bundle.js'></script>\n<script src='https://unpkg.com/vue-chartkick@0.5.1/dist/vue-chartkick.js'></script>\n<script src='https://unpkg.com/element-ui/lib/index.js'></script>\n"

"<link rel='stylesheet' href='https://unpkg.com/element-ui/lib/theme-chalk/index.css'>\n"

true

In [2]:
group_已育 = User.member.marriage("已育")
group_其他 = User.member.marriage(["单身","已婚","恋爱"])

group={
  "已育"=> group_已育, 
  "其他"=> group_其他, 
}

Store["group_marriage"]=group
nil

In [3]:
all = User.where(id:User.member.marriage("已育").id)

group_a = all.occupation_class("个性化办公室")
group_b = all.occupation_class("企事业单位办公室")
group_c = all.occupation_class("商务办公室")
group_d = all.occupation_class("其他")

group={
  "个性化办公室"=> group_a, 
  "企事业单位办公室"=> group_b, 
  "商务办公室"=> group_c, 
  "其他"=> group_d, 
}

Store["group_jobtype"]=group
nil

In [4]:
# 库存数量 >5
# 宝妈购买转化率 >= 20%

product = ProductSpu.where(spu:
ProductSpu.where(status:2).where(spu:Warehouse.unsold.joins(:product_spu).group(ProductSpu.arel_table[:spu]).count.filter {|k,v| v>5 }.map {|k,v| k}).
        map do |x| [x.spu,x.category,
            ratio(Orders.where(member_id:group_已育.id).finish_purchase.item.keep.where(spu:x.spu).sold.size, Orders.where(member_id:group_已育.id).finish_purchase.item.keep.where(spu:x.spu).size),
            ratio(Orders.finish_purchase.item.keep.where(spu:x.spu).sold.size, Orders.finish_purchase.item.keep.where(spu:x.spu).size)]
        end.filter do |x| x[1][1]!="内衣" and x[1][1]!="首饰" and x[1][1]!="其他" and x[2]>0.2 and x[3]>0.15 end.map do |x| x[0] end
)
Store["product_shortlist"]=product
product.size

21

In [5]:
group_marriage = Store["group_marriage"]
item_删除 = group_marriage["已育"].orders.product.item.where(remove_flag:"\x01")
item_购买 = group_marriage["已育"].orders.product.item.keep.sold
item_未购买 = group_marriage["已育"].orders.product.item.keep.unsold
item_发出 = group_marriage["已育"].orders.product.item.keep

nil

In [6]:
data_删除 = item_删除.includes(:product_spu).group(:spu).count.sort do |x,y| -1*(x[1]<=>y[1]) end.to_h
data_购买 = item_购买.includes(:product_spu).group(:spu).count.sort do |x,y| -1*(x[1]<=>y[1]) end.to_h
data_未购买 = item_未购买.includes(:product_spu).group(:spu).count.sort do |x,y| -1*(x[1]<=>y[1]) end.to_h


nil

In [7]:
data = item_发出.includes(:product_spu).group(:spu).count.sort do |x,y| -1*(x[1]<=>y[1]) end.map do |x|
    [x[0],
        [x[1], 
         data_购买[x[0]] || 0,
         data_未购买[x[0]] || 0,
         data_删除[x[0]] || 0,
         (data_购买[x[0]] || 0) / ((data_购买[x[0]] || 0) + (data_未购买[x[0]] || 0)).to_f
    ]]
end.filter do |x| x[1][4]>0.2 end[0,30]

Store["product_top_send"] = ProductSpu.where(spu:data.map {|x| x[0]})
nil

In [8]:
# 按照购买比例排序 Top 30
data = item_发出.includes(:product_spu).group(:spu).count.sort do |x,y| -1*(x[1]<=>y[1]) end.map do |x|
    [x[0],
        [x[1], 
         data_购买[x[0]] || 0,
         data_未购买[x[0]] || 0,
         data_删除[x[0]] || 0,
    ]]
end.sort do |x,y| -1*( (x[1][1]/x[1][0].to_f)<=> (y[1][1]/y[1][0].to_f)) end.filter {|x| x[1][0] >= 10}[0,30]

Store["product_top_purchase"] = ProductSpu.where(spu:data.map {|x| x[0]})
nil

In [9]:
# 主推品
Store["product_featured"] = ProductSpuProperty.featured.product_spu
nil

In [10]:
product_shortlist = Store["product_shortlist"]
product_top_send = Store["product_top_send"] 
product_top_purchase = Store["product_top_purchase"] 
product_featured = Store["product_featured"] 

puts product_shortlist.size
puts product_top_send.size
puts product_top_purchase.size
puts product_featured.size

puts (product_shortlist+product_top_send+product_top_purchase+product_featured).uniq.size

21
30
30
28
74


In [11]:
time {

User.where("valid_flag=true").pmap {|x| 
   enable_cache()
   puts_to_file(render("用户档案",{:user=>User.includes(:orders).find(x.id) }),"output/user/#{x.id}.html")
}
`cp -r output/. ../public/`    

}

1 minutes 51 seconds


In [12]:
time {

group = Store["group_marriage"]
product = Store["product_shortlist"]

index=0;
product_data = ProductSpu.where(status:2).pmap do |x|     
    index+=1
    puts index

    [x.spu,
        [ "<a href='https://console.wetry.shop/productDetail/#{x.id}'>#{x.spu}</a>" + (product.id.index(x.id) ? "(候选)" : "") +
            (x.property.featured.count>0 ? "(主推)" : ""),
     x.product_name,
     x.member_price,
     x.warehouse.where(status:1).count,
     x.picture.keep.url,
     ratio(Orders.finish_purchase.item.keep.where(spu:x.spu).sold.size, Orders.finish_purchase.item.keep.where(spu:x.spu).size),
     ratio(Orders.where(member_id:group["已育"].id).finish_purchase.item.keep.where(spu:x.spu).sold.size, Orders.where(member_id:group["已育"].id).finish_purchase.item.keep.where(spu:x.spu).size),
     Orders.finish_purchase.item.keep.where(spu:x.spu).size,
     Orders.where(member_id:group["已育"].id).finish_purchase.item.keep.where(spu:x.spu).size,
     ratio(Orders.finish_purchase.item.keep.where(spu:x.spu).size,Orders.where(member_id:group["已育"].id).finish_purchase.item.keep.where(spu:x.spu).size),        
     x.picture.keep.size,
     "https://image.wetry.shop/product/spu/#{x.spu}/productDetailPageWxCode.jpg"
    ]] end.to_h
}

Store["product_data"] = product_data
nil

9 minutes 47 seconds


{"10000"=>["<a href='https://console.wetry.shop/productDetail/1'>10000</a>", "季卡", 0.129e3, 0, "", 1.0, 1.0, 2719, 580, 4.69, 0, "https://image.wetry.shop/product/spu/10000/productDetailPageWxCode.jpg"], "10001"=>["<a href='https://console.wetry.shop/productDetail/2'>10001</a>", "半年卡", 0.299e3, 0, "", 1.0, 1.0, 55, 13, 4.23, 0, "https://image.wetry.shop/product/spu/10001/productDetailPageWxCode.jpg"], "10002"=>["<a href='https://console.wetry.shop/productDetail/3'>10002</a>", "年卡", 0.399e3, 0, "", 1.0, 1.0, 336, 72, 4.67, 0, "https://image.wetry.shop/product/spu/10002/productDetailPageWxCode.jpg"], "10071"=>["<a href='https://console.wetry.shop/productDetail/4'>10071</a>", "橙色V领口设计感连衣裙", 0.288e3, 0, "https://image.wetry.shop/product/spu/10071/10071_a.jpg", 0.29, 0.29, 24, 7, 3.43, 6, "https://image.wetry.shop/product/spu/10071/productDetailPageWxCode.jpg"], "10072"=>["<a href='https://console.wetry.shop/productDetail/5'>10072</a>", "白色双排扣连衣裙", 0.318e3, 0, "https://image.wetry.shop/prod